# Reconocimiento de Entidades Nombradas

El objetivo es identificar entidades nombradas en un corpus del area de genetica. Las etiquetas que se buscan indican si se trata de un gen o no. Debe tomarse en cuenta que las entidades nombradas pueden constar de mas de un elemento. Por tanto, se utiliza un etiquetado BIO.

Las etiquetas son como siguen:
1. __B-tag__ indica el inicio de una entidad (de izquierda a derecha).
2. __I-tag__ indica que la palabra pertenece a una entidad etiquetada con B-tag. En este sentido, siempre debe existir una etiqueta B-tag, pero no necesariamente una I-tag.
3. Finalmente la etiqueta __O__ indica que no se trata de una entidad nombrada.

## Cadenas Ocultas de Markov

Para dar solucion al problema propuesto se usara el modelo de aprendizaje automatico Hidden Markov Model (HMM). Es importante establecer que para construir un HMM se necesitan llevar a cabo tres etapas:

1. 
2. Decodificacion
3. Aprendizaje

### Construccion del modelo del lenguaje $\lambda$

Un modelo oculto de Markov se denota mediante la letra $\lambda$ y es una 5-tupla:

$$ \lambda = (S, O, A, B, \Pi) $$

Donde: 

* $S = \{s_1, ..., s_N\}$
* $O = \{o_1, ..., o_T\}$
* $A = \{a_{i,j}\} = p(q_{t+1}=S_j|q_t=S_i)$
* $B = \{b_{i,j}\} = p(q_t=o_i| q_t=Sj)$
* $\Pi = \{\Pi_i\} = p(q_1=S_i)$

In [312]:
import numpy as np
from nltk import bigrams
import re

### Lectura y preprocesamiento del corpus

El conjunto de estados ocultos $S$ se conformara por las etiquetas BIOS y el conjunto de simbolos de observacion $O$ seran las palabras.

In [275]:
with open('Final1/data_test.txt', 'r') as file:
    raw_corpus = file.read().splitlines()

In [288]:
raw_corpus[:10]

['IL-2\tB-DNA',
 'gene\tI-DNA',
 'expression\tO',
 'and\tO',
 'NF-kappa\tB-protein',
 'B\tI-protein',
 'activation\tO',
 'through\tO',
 'CD28\tB-protein',
 'requires\tO']

In [289]:
corpus = {
    'states': [],
    'obs': []
}

S = []
O = []


string_obs = ''
string_state = ''

for phrase in raw_corpus:
    if phrase == '' and len(string_obs) > 0 and len(string_state) > 0:
        corpus['states'].append('<BOS> ' + string_state + ' <EOS>')
        corpus['obs'].append('<BOS> ' + string_obs + ' <EOS>')
        string_obs = ''
        string_state = ''
    try:
        obs , state = phrase.split('\t')
        string_obs += obs + ' ' 
        string_state += state + ' '
        # Creando conjuntos de simbolos
        if obs not in O:
            O.append(obs)
        if state not in S:
            S.append(state)
    except:
        pass

In [305]:
print(len(corpus['obs'][0].split()))
print(corpus['obs'][0].split())

18
['<BOS>', 'IL-2', 'gene', 'expression', 'and', 'NF-kappa', 'B', 'activation', 'through', 'CD28', 'requires', 'reactive', 'oxygen', 'production', 'by', '5-lipoxygenase', '.', '<EOS>']


In [306]:
print(len(corpus['states'][0].split()))
print(corpus['states'][0].split())

18
['<BOS>', 'B-DNA', 'I-DNA', 'O', 'O', 'B-protein', 'I-protein', 'O', 'O', 'B-protein', 'O', 'O', 'O', 'O', 'O', 'B-protein', 'O', '<EOS>']


In [292]:
len_S = len(S)
len_O = len(O)

print('Numero de estados = ', len_S)
print('Alfabeto de observaciones = ', len_O)

Numero de estados =  11
Alfabeto de observaciones =  22053


### Obtener frecuencias de bigramas

In [316]:
freq_states = {}
freq_obs = {}
freq_states2states = {}
freq_states2obs = {}

In [344]:
list(bigrams(corpus['states'][0].split()))

[('<BOS>', 'B-DNA'),
 ('B-DNA', 'I-DNA'),
 ('I-DNA', 'O'),
 ('O', 'O'),
 ('O', 'B-protein'),
 ('B-protein', 'I-protein'),
 ('I-protein', 'O'),
 ('O', 'O'),
 ('O', 'B-protein'),
 ('B-protein', 'O'),
 ('O', 'O'),
 ('O', 'O'),
 ('O', 'O'),
 ('O', 'O'),
 ('O', 'B-protein'),
 ('B-protein', 'O'),
 ('O', '<EOS>')]

In [353]:
for sstate, sobs in zip(corpus['states'], corpus['obs']):
    tags = ['<BOS>', '<EOS>']
    for si, sj in list(bigrams(sstate.split())):
        if (si, sj) in freq_states2states:
            freq_states2states[(si, sj)] += 1
        else:
            freq_states2states[(si, sj)] = 1

    for si in sstate.split():
        if si in freq_states:
            freq_states[si] += 1
        else:
            freq_states[si] = 1
            
    for oi, sj in zip(sobs.split()[1:-1], sstate.split()[1:-1]):
        if (oi, sj) not in freq_states2obs:
            freq_states2obs[(oi, sj)] = 1
        else:
            freq_states2obs[(oi, sj)] += 1

In [381]:
freq_states

{'<BOS>': 148299,
 'B-DNA': 76259,
 'I-DNA': 126187,
 'O': 3062094,
 'B-protein': 241921,
 'I-protein': 198611,
 '<EOS>': 148299,
 'B-cell_type': 53688,
 'I-cell_type': 69872,
 'B-cell_line': 30624,
 'I-cell_line': 59032,
 'B-RNA': 7608,
 'I-RNA': 12240}

In [423]:
(freq_states2states[('<BOS>', S[0])]+1)/(freq_states['<BOS>']+ len_S)

0.009857730429505765

## Creacion del modelo del lenguaje

El modelo del lenguaje sera construido al hacer las matrices $A$, $B$ y $\Pi$, usando los alfabetos $S$ y $O$ obtenidos previamente.

$a_{i,j} = p(q_{t+1}=S_i| q_t=S_j)$

$p(q_{t+1}=S_i| q_t=S_j) = \frac{fr(s_i, s_j) + 1}{fr(sj) + N}$ 

In [358]:
# Inicializacion de matrices
A = np.zeros((len_S, len_S))
B = np.zeros((len_S, len_O))
Pi = np.zeros(len_S)

N = len_S

print(A.shape)
print(B.shape)
print(Pi.shape)

(11, 11)
(11, 22053)
(11,)


In [431]:
def smoothingLaplacian(wi, wj, N, sigma):
    if sigma == 'state2state':
        try:
            return (freq_states2states[(wi, wj)] + 1) / (freq_states[wj] + N)
        except: 
            return 1 / (freq_states[wj] + N)
    elif sigma == 'state2obs':
        try:
            return (freq_states2obs[(wi, wj)] + 1) / (freq_obs[wj] + N)
        except:
            return 1 / (freq_obs[wj] + N)
    elif sigma == 'initial':
        try:
            prob = (freq_states2states[('<BOS>', wi)] + 1) / (freq_states['<BOS>'] + N)
        except:
            prob = 1 / (freq_states['<BOS>'] + N)
            
    return prob

In [436]:
print(smoothingLaplacian(S[2], S[0], N, 'initial'))

1.0067493763063853
